In [ ]:
import os
import random
import numpy as np
from PIL import Image
from torchvision import transforms

# 定義標籤類別對應的索引
CLASS_INDEX = {
    'hat_on': 0,
    'hat_off': 1,
    'clothes_on': 2,
    'clothes_off': 3
}

def cutmix_data(image_paths, labels_a, labels_b, output_folder, alpha=1.0):
    # 隨機選擇兩張圖片
    image_path_a, image_path_b = image_paths[0], image_paths[1]
    
    # 讀取圖片
    img_a = Image.open(image_path_a)
    img_b = Image.open(image_path_b)

    # 轉換成Tensor
    tensor_transform = transforms.ToTensor()
    img_tensor_a = tensor_transform(img_a)
    img_tensor_b = tensor_transform(img_b)

    # CutMix參數
    lam = np.random.beta(alpha, alpha)
    
    # 計算剪裁框的位置
    bbx1, bby1, bbx2, bby2 = rand_bbox(img_tensor_a.size(), lam)

    # 複製圖片
    mixed_img_tensor = img_tensor_a.clone()

    # 將兩張圖片混合
    mixed_img_tensor[:, bby1:bby2, bbx1:bbx2] = img_tensor_b[:, bby1:bby2, bbx1:bbx2].clone()

    # 更新標籤
    mixed_labels = update_labels(labels_a, labels_b, bbx1, bby1, bbx2, bby2, img_tensor_a.size())

    # 儲存混合後的圖片
    output_image_path = os.path.join(output_folder, os.path.basename(image_path_a))
    mixed_img = transforms.ToPILImage()(mixed_img_tensor)
    mixed_img.save(output_image_path)

    # 儲存對應的標籤至標籤檔案
    mixed_label_path = os.path.join(output_folder, os.path.basename(image_path_a).replace('.png', '.txt'))
    with open(mixed_label_path, 'w') as f:
        for label in mixed_labels:
            label_str = ' '.join([str(elem) for elem in label])
            f.write(label_str + '\n')

    return mixed_img, mixed_labels

def rand_bbox(size, lam):
    W = size[2]
    H = size[1]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def update_labels(labels_a, labels_b, bbx1, bby1, bbx2, bby2, image_size):
    new_labels = []

    # 重新計算圖像 A 中的標籤
    for label in labels_a:
        class_index, x_center, y_center, width, height = label
        x_center *= image_size[2]
        y_center *= image_size[1]
        width *= image_size[2]
        height *= image_size[1]

        if bbx1 <= x_center <= bbx2 and bby1 <= y_center <= bby2:
            continue  # 跳過被覆蓋的標籤

        new_x_center = x_center / image_size[2]
        new_y_center = y_center / image_size[1]
        new_width = width / image_size[2]
        new_height = height / image_size[1]
        new_labels.append([class_index, new_x_center, new_y_center, new_width, new_height])

    # 重新計算圖像 B 中的標籤
    for label in labels_b:
        class_index, x_center, y_center, width, height = label
        x_center *= image_size[2]
        y_center *= image_size[1]
        width *= image_size[2]
        height *= image_size[1]

        if not (bbx1 <= x_center <= bbx2 and bby1 <= y_center <= bby2):
            continue  # 跳過未覆蓋的標籤

        new_x_center = (x_center - bbx1) / image_size[2]
        new_y_center = (y_center - bby1) / image_size[1]
        new_width = width / image_size[2]
        new_height = height / image_size[1]
        new_labels.append([class_index, new_x_center, new_y_center, new_width, new_height])

    return new_labels

# 設置資料夾路徑
input_folder = r"C:\Users\Ray\Desktop\宜鼎\c4_second_slice"  # 原始圖片的資料夾路徑
output_folder = "Cutmix"  # 生成的圖片和標籤的資料夾路徑

# 確保輸出資料夾存在，若不存在則創建
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
# 將圖片檔案名稱及其對應的標籤讀取進來
def load_images_and_labels(folder):
    image_paths = []
    labels_dict = {}
    for file in os.listdir(folder):
        if file.endswith('.png'):
            image_path = os.path.join(folder, file)
            image_paths.append(image_path)
            label_path = image_path.replace('.png', '.txt')
            with open(label_path, 'r') as f:
                labels = []
                for line in f:
                    parts = line.strip().split()
                    labels.append([int(parts[0]), float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])])
                labels_dict[image_path] = labels
    return image_paths, labels_dict

# 從資料夾中加載圖片和標籤
image_paths, labels_dict = load_images_and_labels(input_folder)

# 隨機選擇成對的圖片進行 CutMix 操作
for _ in range(len(image_paths) // 2):
    # 隨機選取兩張圖片的路徑
    selected_paths = random.sample(image_paths, 2)
    image_path_a, image_path_b = selected_paths
    labels_a = labels_dict[image_path_a]
    labels_b = labels_dict[image_path_b]

    # 進行 CutMix 操作並保存結果
    mixed_img, mixed_labels = cutmix_data([image_path_a, image_path_b], labels_a, labels_b, output_folder)
    print(f"Processed and saved: {os.path.basename(image_path_a)} and {os.path.basename(image_path_b)}")



In [1]:
import os
import random
import numpy as np
from PIL import Image
from torchvision import transforms

# 定義標籤類別對應的索引
CLASS_INDEX = {
    'hat_on': 0,
    'hat_off': 1,
    'clothes_on': 2,
    'clothes_off': 3
}

def cutmix_data(image_paths, labels_a, labels_b, output_folder, count, alpha=1.0):
    for i in range(count):
        # 隨機選擇兩張圖片
        image_path_a, image_path_b = image_paths[0], image_paths[1]
        
        # 讀取圖片
        img_a = Image.open(image_path_a)
        img_b = Image.open(image_path_b)

        # 轉換成Tensor
        tensor_transform = transforms.ToTensor()
        img_tensor_a = tensor_transform(img_a)
        img_tensor_b = tensor_transform(img_b)

        # CutMix參數
        lam = np.random.beta(alpha, alpha)
        
        # 計算剪裁框的位置
        bbx1, bby1, bbx2, bby2 = rand_bbox(img_tensor_a.size(), lam)

        # 複製圖片
        mixed_img_tensor = img_tensor_a.clone()

        # 將兩張圖片混合
        mixed_img_tensor[:, bby1:bby2, bbx1:bbx2] = img_tensor_b[:, bby1:bby2, bbx1:bbx2].clone()

        # 更新標籤
        mixed_labels = update_labels(labels_a, labels_b, bbx1, bby1, bbx2, bby2, img_tensor_a.size())

        # 儲存混合後的圖片
        output_image_path = os.path.join(output_folder, f"{os.path.basename(image_path_a).replace('.png', '')}_{i}.png")
        mixed_img = transforms.ToPILImage()(mixed_img_tensor)
        mixed_img.save(output_image_path)

        # 儲存對應的標籤至標籤檔案
        mixed_label_path = os.path.join(output_folder, f"{os.path.basename(image_path_a).replace('.png', '')}_{i}.txt")
        with open(mixed_label_path, 'w') as f:
            for label in mixed_labels:
                label_str = ' '.join([str(elem) for elem in label])
                f.write(label_str + '\n')

        print(f"Processed and saved: {output_image_path}")

def rand_bbox(size, lam):
    W = size[2]
    H = size[1]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def update_labels(labels_a, labels_b, bbx1, bby1, bbx2, bby2, image_size):
    new_labels = []

    # 重新計算圖像 A 中的標籤
    for label in labels_a:
        class_index, x_center, y_center, width, height = label
        x_center *= image_size[2]
        y_center *= image_size[1]
        width *= image_size[2]
        height *= image_size[1]

        if bbx1 <= x_center <= bbx2 and bby1 <= y_center <= bby2:
            continue  # 跳過被覆蓋的標籤

        new_x_center = x_center / image_size[2]
        new_y_center = y_center / image_size[1]
        new_width = width / image_size[2]
        new_height = height / image_size[1]
        new_labels.append([class_index, new_x_center, new_y_center, new_width, new_height])

    # 重新計算圖像 B 中的標籤
    for label in labels_b:
        class_index, x_center, y_center, width, height = label
        x_center *= image_size[2]
        y_center *= image_size[1]
        width *= image_size[2]
        height *= image_size[1]

        if not (bbx1 <= x_center <= bbx2 and bby1 <= y_center <= bby2):
            continue  # 跳過未覆蓋的標籤

        new_x_center = (x_center - bbx1) / image_size[2]
        new_y_center = (y_center - bby1) / image_size[1]
        new_width = width / image_size[2]
        new_height = height / image_size[1]
        new_labels.append([class_index, new_x_center, new_y_center, new_width, new_height])

    return new_labels

# 設置資料夾路徑
input_folder = r"C:\Users\Ray\Desktop\宜鼎\c4_second_slice"  # 原始圖片的資料夾路徑
output_folder = "Cutmix"  # 生成的圖片和標籤的資料夾路徑

# 確保輸出資料夾存在，若不存在則創建
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 將圖片檔案名稱及其對應的標籤讀取進來
def load_images_and_labels(folder):
    image_paths = []
    labels_dict = {}
    for file in os.listdir(folder):
        if file.endswith('.png'):
            image_path = os.path.join(folder, file)
            image_paths.append(image_path)
            label_path = image_path.replace('.png', '.txt')
            with open(label_path, 'r') as f:
                labels = []
                for line in f:
                    parts = line.strip().split()
                    labels.append([int(parts[0]), float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])])
                labels_dict[image_path] = labels
    return image_paths, labels_dict

# 從資料夾中加載圖片和標籤
image_paths, labels_dict = load_images_and_labels(input_folder)

# 總共需要生成的圖片數量
total_images_to_generate = 2000

# 每對圖片進行的 CutMix 次數
cutmix_per_pair = total_images_to_generate // (len(image_paths) // 2)

# 隨機選擇成對的圖片進行 CutMix 操作
for _ in range(len(image_paths) // 2):
    # 隨機選取兩張圖片的路徑
    selected_paths = random.sample(image_paths, 2)
    image_path_a, image_path_b = selected_paths
    labels_a = labels_dict[image_path_a]
    labels_b = labels_dict[image_path_b]

    # 進行 CutMix 操作並保存結果
    cutmix_data([image_path_a, image_path_b], labels_a, labels_b, output_folder, cutmix_per_pair)


Processed and saved: Cutmix\innodisk_999_0.png
Processed and saved: Cutmix\innodisk_999_1.png
Processed and saved: Cutmix\innodisk_1346_0.png
Processed and saved: Cutmix\innodisk_1346_1.png
Processed and saved: Cutmix\innodisk_1278_0.png
Processed and saved: Cutmix\innodisk_1278_1.png
Processed and saved: Cutmix\innodisk_1105_0.png
Processed and saved: Cutmix\innodisk_1105_1.png
Processed and saved: Cutmix\innodisk_1017_0.png
Processed and saved: Cutmix\innodisk_1017_1.png
Processed and saved: Cutmix\innodisk_772_0.png
Processed and saved: Cutmix\innodisk_772_1.png
Processed and saved: Cutmix\innodisk_582_0.png
Processed and saved: Cutmix\innodisk_582_1.png
Processed and saved: Cutmix\innodisk_1111_0.png
Processed and saved: Cutmix\innodisk_1111_1.png
Processed and saved: Cutmix\innodisk_1330_0.png
Processed and saved: Cutmix\innodisk_1330_1.png
Processed and saved: Cutmix\innodisk_391_0.png
Processed and saved: Cutmix\innodisk_391_1.png
Processed and saved: Cutmix\innodisk_826_0.png
P